In [ ]:
import os

print(os.getcwd())

from ..utility.src.plot import linestyle_generator
from vertical_drive_arm_model import VerticalDriveArm

ImportError: attempted relative import with no known parent package